# PRACTICA 2

## 0. Importacion de Modulos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import re
import unicodedata
from difflib import SequenceMatcher


spanish_stop_words = stopwords.words('spanish')

In [ ]:
def txt_to_list(filename):
    lines_list = []
    with open(filename, 'r') as file:
        for line in file:
            lines_list.append(line.strip()) # .strip() removes leading/trailing whitespace and newlines
    return lines_list



## 1. Data Wrangling


Los datos se obtuvieron de un chat de WhatsApp, posteriormente se exportaron a un TXT.

In [ ]:
import re
import pandas as pd
from datetime import datetime

RUTA_ARCHIVO = "datos/Datos.txt"

PATRON_MENSAJE = re.compile(
    r'^(\d{1,2}/\d{1,2}/\d{4}), (\d{1,2}:\d{2})\s?(a\.?\s?m\.?|p\.?\s?m\.?) - (.*?): (.*)'
)


with open(RUTA_ARCHIVO, encoding="utf-8") as f:
    lineas = f.readlines()

mensajes = []
mensaje_actual = None

for linea in lineas:
    linea = linea.strip()

    match = PATRON_MENSAJE.match(linea)

    if match:
        # Guardar mensaje anterior
        if mensaje_actual:
            mensajes.append(mensaje_actual)

        fecha, hora, am_pm, autor, texto = match.groups()

        mensaje_actual = {
            "Fecha": fecha,
            "Hora": hora,
            "AM_PM": am_pm.lower(),
            "Autor": autor,
            "Mensaje": texto
        }
    else:
        # Línea adicional (mensaje multilínea)
        if mensaje_actual:
            mensaje_actual["Mensaje"] += " " + linea

# Agregar último mensaje
if mensaje_actual:
    mensajes.append(mensaje_actual)

# =========================
# 4. CREACIÓN DEL DATAFRAME
# =========================

df_original = pd.DataFrame(mensajes)

# =========================
# 5. LIMPIEZA BÁSICA
# =========================

# Eliminar mensajes del sistema
df_original = df_original[~df_original["Autor"].str.contains("mensajes y las llamadas|creó el grupo|te añadió", case=False, na=False)]

# Detectar multimedia
df_original["Tiene_Multimedia"] = df_original["Mensaje"].str.contains("Multimedia omitido|archivo adjunto", case=False).astype(int)

# Limpiar texto multimedia
df_original["Mensaje"] = df_original["Mensaje"].replace(
    to_replace=r"<Multimedia omitido>|archivo adjunto",
    value="",
    regex=True
)

# =========================
# 6. CONVERSIÓN DE FECHA Y HORA
# =========================

def convertir_hora(hora, am_pm):
    hora_dt = datetime.strptime(hora, "%H:%M")
    h = hora_dt.hour

    if am_pm.startswith("p") and h != 12:
        h += 12
    if am_pm.startswith("a") and h == 12:
        h = 0

    return h + hora_dt.minute / 60


df_original["Fecha"] = pd.to_datetime(df_original["Fecha"], format="%d/%m/%Y", errors="coerce")
df_original["Hora_Num"] = df_original.apply(lambda x: convertir_hora(x["Hora"], x["AM_PM"]), axis=1)

# =========================
# 7. FEATURES BÁSICAS DE TEXTO
# =========================

df_original["Num_Caracteres"] = df_original["Mensaje"].str.len()
df_original["Num_Palabras"] = df_original["Mensaje"].str.split().str.len()
df_original["Signo_Pregunta"] = df_original["Mensaje"].str.contains(r"\?", regex=True).astype(int)

# =========================
# 8. RESULTADO FINAL
# =========================

df_original = df_original.reset_index(drop=True)

print(df_original.head())
print("\nTotal de mensajes procesados:", len(df_original))

df_original.to_csv("datos_limpios.csv")

In [ ]:
df_original.info()


In [ ]:
PATRON_EMOJIS = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticonos
    "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
    "\U0001F680-\U0001F6FF"  # transporte y mapas
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FAFF"
    "\u2600-\u26FF"          # símbolos varios
    "\u2700-\u27BF"
    "]+",
    flags=re.UNICODE
)


In [ ]:
df_original["Mensaje"] = df_original["Mensaje"].str.replace(
    PATRON_EMOJIS,
    "",
    regex=True
)

In [ ]:
df = df_original.copy()

In [ ]:
import numpy as np

# 1. Convertir posibles valores nulos o vacíos en la columna Mensaje
# Esto limpia espacios en blanco y convierte celdas vacías en NaN
df['Mensaje'] = df['Mensaje'].astype(str).str.strip()
df['Mensaje'] = df['Mensaje'].replace(['', 'nan', 'None'], np.nan)

# 2. Eliminar las filas donde el Mensaje es nulo
# Usamos inplace=True para que los cambios se guarden en el mismo DataFrame
df.dropna(subset=['Mensaje'], inplace=True)

# 3. Opcional: Filtrar filas donde Num_Palabras sea 0
# (Como se ve en tus índices 3 y 4 de la imagen)
df = df[df['Num_Palabras'] > 0]

# Verificamos cuántas filas quedaron
print(f"Registros después de la limpieza: {len(df)}")
df.head()

In [ ]:
df["Mensaje"].str.extract(r"\.([^.]+)$")[0].unique()


## 2. Solucion Analıtica

### 2.1 Variable objetivo discreta

Esta variable se utiliza para identificar de manera clara el tema principal de cada mensaje, diferenciando aquellos relacionados con plantas carnívoras de los mensajes sobre otros asuntos. Su utilidad principal es medir el grado de enfoque del chat y evaluar si la conversación cumple con el propósito para el cual fue creado.

In [ ]:
palabras_tecnicas = txt_to_list('datos/palabras_tecnicas.txt')
print("Numero de palabras:", len(palabras_tecnicas))

In [ ]:
def mensaje_basura(texto):
    if pd.isna(texto) or not isinstance(texto, str):
        return True

    if any(k in texto.lower() for k in ['http', 'www', 'com/', '.com', 'share/']):
        return True
    if any(ext in texto.lower() for ext in ['.jpg', '.webp', '.png', 'jpg ()', 'webp ()']):
        return True
    limpio = re.sub(r'[^a-zA-Z\s]', '', texto).strip()
    if len(limpio) < 2:
        return True

    return False

def normalizar_texto(texto):
    if pd.isna(texto): return ""


    texto = texto.lower()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    texto = re.sub(r'\b(xd|ok|si|no)\b', '', texto)
    texto = re.sub(r'(j|a|x){2,}', '', texto)
    texto = re.sub(r'[^a-z\s]', '', texto)

    return ' '.join(texto.split())

def calcular_similitud(a, b):
    return SequenceMatcher(None, a, b).ratio()

def etiquetar_mensaje(mensaje):
    texto_limpio = normalizar_texto(mensaje)
    palabras_mensaje = texto_limpio.split()

    for palabra_m in palabras_mensaje:
        for palabra_t in palabras_tecnicas:
            if calcular_similitud(palabra_m, palabra_t) >= 0.90:
                return 1
    return 0

# Aplicar al DataFrame
print("Registros antes de filtrar basura:", len(df))
df["Mensaje"] = df["Mensaje"].apply(normalizar_texto)
df = df[df['Mensaje'].apply(lambda x: not mensaje_basura(x))]
print("Registros despues de filtrar basura:", len(df))

#df["Tipo_Mensaje"] = df["Mensaje"].apply(etiquetar_mensaje)

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from concurrent.futures import ThreadPoolExecutor


model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings_tecnicos = model.encode(
    palabras_tecnicas,
    normalize_embeddings=True
)

def procesar_chunk(textos, threshold=0.75):
    embeddings = model.encode(
        textos,
        normalize_embeddings=True,
        batch_size=32,
        show_progress_bar=False
    )

    sims = cosine_similarity(embeddings, embeddings_tecnicos)
    max_sims = sims.max(axis=1)

    return (max_sims >= threshold).astype(int)


def etiquetar_con_hilos(textos, chunk_size=256, max_workers=4):
    resultados = [0] * len(textos)

    chunks = [
        (i, textos[i:i + chunk_size])
        for i in range(0, len(textos), chunk_size)
    ]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(procesar_chunk, chunk): idx
            for idx, chunk in chunks
        }

        for future in tqdm(
            as_completed(futures),
            total=len(futures),
            desc="Procesando chunks",
            unit="chunk"
        ):
            idx = futures[future]
            res = future.result()
            resultados[idx:idx + len(res)] = res

    return resultados


print("Registros antes de filtrar basura:", len(df))

df = df[df["Mensaje"].apply(lambda x: not mensaje_basura(x))]
df["Mensaje"] = df["Mensaje"].apply(normalizar_texto)

print("Registros después de limpiar:", len(df))

df["Tipo_Mensaje"] = etiquetar_con_hilos(
    df["Mensaje"].tolist(),
    chunk_size=256,
    max_workers=4
)


In [ ]:
df.to_csv("datos/variable_discreta.csv", index=False)

Clasificación Tipo de Mensaje

In [ ]:
temas = {
    "tecnico": [
        "planta", "sustrato", "purpurea", "carnivora", "hijuelos", "cultivo", "raíz"
    ],
    "social": [
        "hola", "adios", "jaja", "😂", "🤣", "amigos", "gracias", "buenas noches", "buenos días"
    ],
    "organizativo": [
        "reunion", "evento", "fecha", "lugar", "hora", "proximo", "organizar"
    ]
}

In [ ]:
def clasificar_tema(texto):
    texto = texto.lower()
    for tema, palabras in temas.items():
        if any(p in texto for p in palabras):
            return tema
    return "otro"

df["Tema_Mensaje"] = df["Mensaje"].apply(clasificar_tema)

In [ ]:
print(df["Tema_Mensaje"].value_counts())
print(df[["Mensaje", "Tema_Mensaje"]].head(10))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Tipo_Mensaje'].unique()

In [ ]:
df['Tipo_Mensaje'].hist()
### Como se puede observar el dataset esta desbalanceado

In [ ]:
# Preparacion de Datos
X = df["Mensaje"]
y = df["Tipo_Mensaje"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

vectorizador = TfidfVectorizer(
    stop_words=spanish_stop_words,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9,
    lowercase=True
)

X_train_tfidf = vectorizador.fit_transform(X_train)
X_test_tfidf = vectorizador.transform(X_test)

### 2.1.2 Entrenamiento de Modelo

## 2.2 Variable Continua

Esta variable permite medir qué tan técnico o especializado es el contenido de cada mensaje mediante una escala gradual, en lugar de clasificarlo solo como “básico” o “avanzado”. Es útil para evaluar la calidad del contenido compartido, identificar mensajes con mayor aporte de conocimiento y entender el nivel general de experiencia presente en el chat. Desde una perspectiva práctica, ayuda a detectar a los usuarios más experimentados, promover contenido de mayor valor y comprender si la comunidad está funcionando como un espacio de aprendizaje y especialización.

### Regresion Logistica

### Tratanto el Desbalanceo

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

log_reg = LogisticRegression(max_iter=1000,solver='saga',  class_weight='balanced')


## Ajuste de Hiperparametros
param_dist = {
    'C': loguniform(1e-4, 1e2),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
}

random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_dist,
    n_iter=30,
    scoring='f1',
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train_tfidf, y_train)

## Mejor Modelo
modelo = random_search.best_estimator_

print("Mejores hiperparámetros:")
print(random_search.best_params_)



In [ ]:
# Evaluacion
from sklearn.metrics import classification_report, confusion_matrix

y_pred = modelo.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


### Sin Tratar el Desbalanceo

In [ ]:
# -----------------------------
# Sin tratar desbalance
# -----------------------------
log_reg = LogisticRegression(
    max_iter=1000,
    solver='saga'

)


param_dist = {
    'C': loguniform(1e-4, 1e2),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
}


random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_dist,
    n_iter=30,
    scoring='f1',
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train_tfidf, y_train)


modelo_sin_desbalanceo = random_search.best_estimator_

print("Mejores hiperparámetros:")
print(random_search.best_params_)

In [ ]:
# Evaluacion
from sklearn.metrics import classification_report, confusion_matrix

y_pred = modelo_sin_desbalanceo .predict(X_test_tfidf)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

### Palabras Asociadas a mensajes Tecnicos

In [ ]:
import numpy as np

feature_names = vectorizador.get_feature_names_out()
coeficientes = modelo.coef_[0]

top_palabras = sorted(
    zip(feature_names, coeficientes),
    key=lambda x: x[1],
    reverse=True
)[:100]

print("Palabras más asociadas a mensajes técnicos:")
for palabra, peso in top_palabras:
    print(palabra, round(peso, 3))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Matriz de confusión normalizada por fila
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Visualización
sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Greens",
            xticklabels=modelo.classes_, yticklabels=modelo.classes_)
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión Normalizada (%)")
plt.show()

### Maquinas de Soporte Vectorial

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(
    class_weight='balanced',
    max_iter=2000
)

# Ajuste de Hiperparametro
param_dist = {
    'C': loguniform(1e-4, 1e2),
    'loss': ['hinge', 'squared_hinge']
}

random_search_svm = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_dist,
    n_iter=40,
    scoring='f1',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search_svm.fit(X_train_tfidf, y_train)

# Mejor modelo
modelo_svm = random_search_svm.best_estimator_

print("Mejores hiperparámetros (SVM):")
print(random_search_svm.best_params_)

In [ ]:
### Evaluacion
y_pred = modelo.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## 2.2 Modelo Variable Objetivo Continua

In [ ]:
ANCHOR_EXPERT = [
    "Nepenthes villosa es una especie de alta montaña que requiere noches frías, alta humedad y excelente oxigenación radicular",
    "El peristoma de Nepenthes funciona como una superficie resbaladiza cuando está húmedo, facilitando la captura de presas",
    "Las Nepenthes de tierras altas presentan metabolismo adaptado a temperaturas nocturnas bajas",
    "El sustrato mineral con buen drenaje reduce el riesgo de pudrición radicular en Nepenthes",
    "Sarracenia presenta rizomas subterráneos y requiere un periodo de dormancia invernal para un crecimiento saludable",
    "Las Sarracenia obtienen nutrientes principalmente de insectos atrapados en sus ascidios",
    "La falta de dormancia puede debilitar progresivamente a las Sarracenia",
    "Drosera captura presas mediante mucílago producido por tricomas glandulares",
    "La digestión en Drosera ocurre mediante enzimas secretadas sobre la presa atrapada",
    "Pinguicula presenta hojas con glándulas pegajosas que permiten la captura de pequeños insectos",
    "Algunas especies de Pinguicula desarrollan hojas no carnívoras durante la estación seca",
    "Los híbridos interespecíficos de Nepenthes pueden mostrar dominancia fenotípica del peristoma",
    "La identificación taxonómica de Nepenthes se basa en características como el peristoma, la tapa y el indumento",
    "Existen complejos de especies en Nepenthes que dificultan su clasificación taxonómica",
    "El exceso de humedad estancada favorece infecciones fúngicas en plantas carnívoras",
    "La ventilación constante es clave para evitar patógenos en cultivos de Nepenthes",
    "El uso de agua con baja conductividad es fundamental para el cultivo de plantas carnívoras",
    "Muchas plantas carnívoras habitan suelos pobres en nutrientes, lo que explica su estrategia carnívora",
    "La captura de insectos permite a las plantas carnívoras suplementar nitrógeno y fósforo"
]


In [ ]:
ANCHOR_CASUAL = [
    "qué bonita planta",
    "me gusta mucho esa planta",
    "se ve bien chula",
    "está preciosa",
    "me encantan las plantas",
    "esa planta está rara",
    "nunca había visto una así",
    "yo quiero una",
    "dónde la compraste",
    "estoy imprimiendo una pieza en 3D",
    "mi impresora 3D falló otra vez",
    "ya terminé de imprimir el soporte",
    "estoy diseñando una pieza en fusion 360",
    "esa pieza quedó perfecta en PLA",
    "voy a soldar unos cables",
    "el ventilador ya no prende",
    "tengo que cambiar el relay",
    "el controlador dejó de funcionar",
    "buenos días",
    "jajaja",
    "no inventes",
    "qué onda",
    "gracias"
]


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

expert_vec = embedder.encode(
    ANCHOR_EXPERT,
    normalize_embeddings=True
).mean(axis=0)

casual_vec = embedder.encode(
    ANCHOR_CASUAL,
    normalize_embeddings=True
).mean(axis=0)


In [ ]:
from numpy.linalg import norm

def specialization_score_embedding(text: str) -> float:
    if not isinstance(text, str) or text.strip() == "":
        return 0.0

    vec = embedder.encode(text, normalize_embeddings=True)

    sim_expert = np.dot(vec, expert_vec)
    sim_casual = np.dot(vec, casual_vec)

    score = (sim_expert - sim_casual + 1) / 2
    return round(np.clip(score, 0, 1), 3)


In [ ]:
df["especializacion"] = df["Mensaje"].apply(specialization_score_embedding)


In [ ]:
df

In [ ]:
df.to_csv("datos_etiquetados1.csv")

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np


# Cargar modelo
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Vectores promedio de referencia
expert_vec = embedder.encode(ANCHOR_EXPERT, normalize_embeddings=True).mean(axis=0)
casual_vec = embedder.encode(ANCHOR_CASUAL, normalize_embeddings=True).mean(axis=0)

# Procesar todos los mensajes en lote
mensajes = df["Mensaje"].tolist()
embeddings = embedder.encode(mensajes, batch_size=32, show_progress_bar=True)

# Calcular puntajes continuos en lote
sim_expert = np.dot(embeddings, expert_vec)
sim_casual = np.dot(embeddings, casual_vec)
scores = (sim_expert - sim_casual + 1) / 2

# Guardar en el DataFrame con dos decimales
df["especializacion"] = np.clip(scores, 0, 1).round(2)

In [ ]:
df

In [ ]:
df.to_csv("Datos_Etiquetados.csv", index=False)